In [1]:
import torch
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os, h5py
from sgg_benchmark.modeling.detector import build_detection_model
from sgg_benchmark.config import cfg

config_file = '/home/maelic/Documents/PhD/MyModel/Scene-Graph-Benchmark-Cuda11.7/configs/VG150/curated/e2e_relation_X_101_32_8_FPN_1x_custom_pred.yaml'
path = "/home/maelic/Documents/Scene-Graph-Benchmark-Cuda11.7/datasets/VG150/curated/VG-SGG-with-attri.h5"
# load with h5py
data = h5py.File(path, 'r')
# load image data
image_data = json.load(open("/home/maelic/Documents/PhD/MyModel/Scene-Graph-Benchmark-Cuda11.7/datasets/vg/image_data.json"))

In [2]:
# get images ids from test split
test_ids = []
for i, img in enumerate(data['split_rel']):
    if img == 2:
        id = image_data[i]['image_id']
        test_ids.append(id)
print(len(test_ids))

32422


In [20]:
cfg.merge_from_file(config_file)

model = build_detection_model(cfg)
model.to(cfg.MODEL.DEVICE)
dummy_input = torch.randn(1, 3, 600, 800)
shape_of_first_layer = list(model.parameters())[0].shape #shape_of_first_layer

N,C = shape_of_first_layer[:2]
print(N)
print(C)
dummy_input = torch.Tensor(N,C)

dummy_input = dummy_input[...,:, None,None] #adding the None for height and weight
torch.onnx.export(model, "sgdet_motifs_tde.onnx", dummy_input)



2023-03-04 14:31:00.400 | INFO     | sgg_benchmark.data.build:get_dataset_statistics:30 - ----------------------------------------------------------------------------------------------------
2023-03-04 14:31:00.400 | INFO     | sgg_benchmark.data.build:get_dataset_statistics:31 - get dataset statistics...
2023-03-04 14:31:00.401 | INFO     | sgg_benchmark.data.build:get_dataset_statistics:42 - Loading data statistics from: /home/maelic/Documents/Scene-Graph-Benchmark-Cuda11.7/checkpoints/VG150/curated/sgdet_motifs_causal_tde/VG150_curated_filtered_train_statistics.cache
2023-03-04 14:31:00.401 | INFO     | sgg_benchmark.data.build:get_dataset_statistics:43 - ----------------------------------------------------------------------------------------------------


loading word vectors from /home/maelic/glove/glove.6B.200d.pt
__background__ -> __background__ 
fail on __background__
loading word vectors from /home/maelic/glove/glove.6B.200d.pt
__background__ -> __background__ 
fail on __background__
64
3


TypeError: Unsupported type for to_image_list: <class 'str'>

In [15]:
print(list(model.parameters())[0].shape)
print(image.shape)

torch.Size([64, 3, 7, 7])
torch.Size([3, 600, 800])


In [ ]:
model.eval()
results_dict = {}
cpu_device = torch.device("cpu")
torch.cuda.empty_cache()
from torchvision.io import read_image

with torch.no_grad():

    image = read_image("/home/maelic/Documents/PhD/Datasets/VisualGenome/VG_100K/10000.jpg")

    output = model(image.to(device), targets)

    output = [o.to(cpu_device) for o in output]